# Deploy Trigger Funtion - `run_upload`

Reference: 
- [tutorials/storage](https://cloud.google.com/functions/docs/tutorials/storage)
- [Cloud SDK on Colab](https://colab.research.google.com/github/tensorflow/cloud/blob/master/g3doc/tutorials/google_cloud_project_setup_instructions.ipynb#scrollTo=WPew3GoCcdtm)

In [1]:
GCP_PROJECT_ID = 'aiffel-gn-3'
PROJECT_NUMBER = '513837032574'

In [2]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
# !gcloud components update


ERROR: (gcloud.components.update) The components management action could not be performed because the installation root of the Cloud SDK could not be located. If you previously used the Cloud SDK installer, you could re-install the SDK and retry again.


In [3]:
!gcloud auth login


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=aBMKN4SKow6wA65BzrsD17r9ZN0TWA&prompt=consent&access_type=offline&code_challenge=ZgIKG1hwChV-XBKov05vhRgA5nPIBQBXjPHP1JGQgKE&code_challenge_method=S256

Enter verification code: 4/1AX4XfWjgdhmwtwK285f_K7f3fEIZL0MvdIsOMMIAlW5a2Qi1RmXi_CsCPNc

You are now logged in as [jieun.jeon.818@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [4]:
!gcloud config set project aiffel-gn-3

Updated property [core/project].


In [ ]:
!gsutil mb gs://gs-test2

Creating gs://gs-test2/...
ServiceException: 409 A Cloud Storage bucket named 'gs-test2' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [11]:
%cd ../

/content


In [12]:
!rm -rf GCF-run-upload/

In [13]:
!git clone https://github.com/aiffel-socar-cv/GCF-run-upload.git

Cloning into 'GCF-run-upload'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 36 (delta 12), reused 31 (delta 7), pack-reused 0
Unpacking objects: 100% (36/36), done.


In [14]:
%cd GCF-run-upload

/content/GCF-run-upload


In [15]:
!ls

main.py  README.md  requirements.txt  sample_images


## Deploy `run_upload` function to Cloud Function

In [ ]:
!gcloud functions deploy run_upload \
--runtime python39 \
--trigger-resource images-original \
--trigger-event google.storage.object.finalize

Created .gcloudignore file. See `gcloud topic gcloudignore` for details.

For Cloud Build Logs, visit: https://console.cloud.google.com/cloud-build/builds;region=us-central1/d762e934-a184-4ceb-8785-e812ae367681?project=513837032574


## Test - uploading an empty .txt file
Checking if the function is triggered correctly when new txt file is uploaded to `images-original` bucket

run below commands under `trigger-run-upload/` directory. (`gcf-test.txt`)

In [ ]:
str = "originals/test.jpg"
tmp = str.split("/")
tmp[-1]

'test.jpg'

In [10]:
!gsutil cp sample_images/input/img_0253.png gs://images-original/originals

Copying file://sample_images/input/img_0253.png [Content-Type=image/png]...
/ [1 files][173.1 KiB/173.1 KiB]                                                
Operation completed over 1 objects/173.1 KiB.                                    


In [ ]:
!gcloud functions logs read --limit 50

LEVEL  NAME        EXECUTION_ID  TIME_UTC                 LOG
D      run_upload  9n7nxbpiiqcn  2021-12-07 06:51:00.536  Function execution took 6260 ms, finished with status: 'crash'
E      run_upload  9n7nxbpiiqcn  2021-12-07 06:51:00.535  Traceback (most recent call last):
                                                            File "/layers/google.python.pip/pip/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
                                                              response = self.full_dispatch_request()
                                                            File "/layers/google.python.pip/pip/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
                                                              rv = self.handle_user_exception(e)
                                                            File "/layers/google.python.pip/pip/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
             

# Check the Database to see a new row is created succesfully

In [ ]:
!gcloud sql connect viai-images --user=postgres --quiet

Connecting to database with SQL user [postgres].Password for user postgres: 

In [ ]:
!sudo apt-get -y -qq install postgresql